In [1]:
%pip install matplotlib seaborn plotly plotnine

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:

# %matplotlib widget
import matplotlib.pyplot as plt
from polygonal_roadmaps import cli, environment
import networkx as nx
import numpy as np
import logging
import pandas as pd
import seaborn as sns
from IPython.display import HTML
import plotly.express as px
import plotly.graph_objects as go
import tempfile
import plotly.express as px

logging.getLogger().setLevel(logging.INFO)

%load_ext snakeviz

In [3]:
scenarios = ["MAPF;room-32-32-4;even", "MAPF;empty-8-8.map;even", "DrivingSwarm;icra2024.yaml;icra2024_1m.yml"]
env = cli.env_generator(scenarios[-1], n_agents=4)[0]
env.state
planner = cli.create_planner_from_config_file("benchmark/planner_config/CCRv2.yml", env)
plan = planner.create_plan()
plan.as_state_list()

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.shared_paths(a, b, **kwargs)
/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning: invalid value encountered in shared_paths
  return lib.sha

[(7, 4, 10, 3),
 (8, 4, 13, 5),
 (7, 4, 13, 12),
 (8, 9, 13, 12),
 (3, 10, 7, 12),
 (5, 13, 8, 9),
 (12, 7, 3, 10),
 (12, None, None, None),
 (12, None, None, None),
 (9, None, None, None),
 (4, None, None, None)]

In [4]:
data = []
for t, state in enumerate(plan.as_state_list()):
    for i, n in enumerate(state):
        #print(f"{t}: {n}")
        if n is None:
            x,y = np.nan, np.nan
        else:
            x,y,*_ = env.g.nodes[n]['pos']
        data.append({"agent": i, "t": t, "node": n, "x": x, "y": y, "done": n is None})
df = pd.DataFrame(data)
df.agent = df.agent.astype('category')
df

,agent,t,node,x,y,done
0,0,0,7.0,0.25,-1.25,False
1,1,0,4.0,2.25,0.75,False
2,2,0,10.0,0.25,0.75,False
3,3,0,3.0,2.25,-1.25,False
4,0,1,8.0,1.25,-1.25,False
5,1,1,4.0,2.25,0.75,False
6,2,1,13.0,0.25,-0.25,False
7,3,1,5.0,2.25,-0.25,False
8,0,2,7.0,0.25,-1.25,False
9,1,2,4.0,2.25,0.75,False


In [5]:
import plotly.graph_objects as go


#df = pd.DataFrame(dict(x=[0], y=[0]))
fig = px.scatter(df, x="x", y="y", animation_frame='t', animation_group="agent", color="agent", hover_name="agent")
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')))
showlegend = True
#fig.update_layout(showlegend=False)
obs = env._obstacles.geoms
for o in obs:
    xy = o.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(20,20,20,0.4)", name="Obstacles", showlegend=showlegend, mode="text"))
    showlegend = False
    
inner = [n['geometry'].inner for _, n in env.g.nodes(data=True)]
showlegend = True

for i in inner:
    xy = i.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,80,0.2)", name="Inner", showlegend=showlegend, mode="text"))
    showlegend = False
    
inner = [n['geometry'].borderPoly for *_, n in env.g.edges(data=True)]
showlegend = True
for i in inner:
    xy = i.exterior.xy
    fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,40,0.05)", name="Transition", showlegend=showlegend, mode="text"))
    showlegend = False
fig.update_layout(width=1000, height=800)
fig.show()

In [8]:
from polygonal_roadmaps.cli import aggregate_results

df = aggregate_results("cluster_results")
df.loc[df["problem_parameters.conflict_horizon"].isna(), "problem_parameters.conflict_horizon"]= 0
df["problem_parameters.conflict_horizon"] = df["problem_parameters.conflict_horizon"].astype('category')

TypeError: 'NoneType' object does not support item assignment

In [ ]:
df.columns

Index(['astar', 'finished', 'flowtime', 'spacetime_astar', 'steps',
       'sum_of_cost', 'valid', 'history', 'config.index', 'config.planner',
       'config.planner_args.priority_method', 'config.planner_file',
       'config.problem_parameters', 'config.scen', 'config.xindex',
       'problem_parameters.conflict_horizon',
       'problem_parameters.k_robustness', 'problem_parameters.pad_path',
       'problem_parameters.step_num', 'problem_parameters.wait_action_cost',
       'problem_parameters.weight_name', 'config.planner_args.max_iter'],
      dtype='object')

In [ ]:
df["problem_parameters.conflict_horizon"].unique()

[0, 5, 3]
Categories (3, int64): [0, 3, 5]

In [ ]:
df.history = df.history.astype(str).astype('category')
px.box(df.loc[df.valid], x="config.planner_file", y="steps", facet_col="config.scen", facet_row="problem_parameters.step_num", hover_name="history", color="problem_parameters.conflict_horizon")

In [ ]:
df_agg = df.groupby(["config.planner_file", "config.scen", "problem_parameters.step_num", "problem_parameters.conflict_horizon"]).valid.sum().reset_index()
px.scatter(df_agg, x="config.planner_file", y="valid", facet_col="config.scen", facet_row="problem_parameters.step_num", color="problem_parameters.conflict_horizon")

In [ ]:
df_agg = df.groupby(["config.planner_file", "config.scen", "problem_parameters.step_num", "problem_parameters.conflict_horizon"]).finished.sum().reset_index()
px.scatter(df_agg, x="config.planner_file", y="finished", facet_col="config.scen", facet_row="problem_parameters.step_num", color="problem_parameters.conflict_horizon")

In [ ]:
def show_run(history_file, scenario):
    df = pd.read_feather(history_file)
    fig = px.scatter(df, x="x", y="y", animation_frame='t', animation_group="agent", color="agent", hover_name="agent")
    fig.update_traces(marker=dict(size=12,
                                  line=dict(width=2,
                                            color='DarkSlateGrey')))
    showlegend = True
#fig.update_layout(showlegend=False)
    env = cli.env_generator(scenario, n_agents=1)[0]
    obs = env._obstacles.geoms
    for o in obs:
        xy = o.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(20,20,20,0.4)", name="Obstacles", showlegend=showlegend, mode="text"))
        showlegend = False
        
    inner = [n['geometry'].inner for _, n in env.g.nodes(data=True)]
    showlegend = True

    for i in inner:
        if i.geom_type != "Polygon":
            continue
        xy = i.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,80,0.2)", name="Inner", showlegend=showlegend, mode="text"))
        showlegend = False
        
    inner = [n['geometry'].borderPoly for *_, n in env.g.edges(data=True)]
    showlegend = True
    for i in inner:
        if i.geom_type != "Polygon":
            continue
        xy = i.exterior.xy
        fig.add_trace(go.Scatter(x=list(xy[0]), y=list(xy[1]), fill="toself", fillcolor="rgba(0,100,40,0.05)", name="Transition", showlegend=showlegend, mode="text"))
        showlegend = False
    fig.update_layout(width=1000, height=800)
    fig.show()


#show_run(df.iloc[22]["history"], df.iloc[22]["config.scen"])
show_run("results/CCRv2.yml/DrivingSwarm;icra2024.yaml;icra2024.yml/0/history.feather", "DrivingSwarm;icra2024.yaml;icra2024.yml")


/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarning:

invalid value encountered in shared_paths

/home/semai/.local/lib/python3.10/site-packages/shapely/linear.py:174: RuntimeWarni

In [ ]:
pd.read_feather("cluster_results/CCRv2.yml/DrivingSwarm;icra2024.yaml;icra2024_1m.yml/1/history.feather").head()

FileNotFoundError: [Errno 2] No such file or directory: 'cluster_results/CCRv2.yml/DrivingSwarm;icra2024.yaml;icra2024_1m.yml/1/history.feather'